# Extract and clean regionalizacao

Neste notebook extraímos os dados de regionalização da LOA e fazemos a limpeza dos dados.

In [1]:
import pandas as pd
from typing import Optional
import plotly.graph_objects as go
import os

from utils.download_file import download_csv_as_dataframe

from config import DATA_FOLDER, GRAFICOS_FOLDER 

pd.set_option('display.max_columns', None)

## Extração dos dados

In [2]:
class DownloadLoaRegionalizada:

    base_url = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads'

    files = {

        2025 : 'LOA.25_VersionamentoDetalhamentoAcao.csv',
        2024 : 'DAAprovado2024.csv',
        2023 : 'DAAprovado2023.csv',
        2022 : 'DAAprovado2022.csv'
    }

    def __build_url(self, year: int) -> str:
        """
        Builds the complete URL for downloading the LOA file for a given year.

        Args:
            year (int): The year for which the LOA file URL is to be built.

        Returns:
            str: The complete URL as a string.
        """
        endpoint = f'{year}/{self.files[year]}'
        return self.base_url + '/' + endpoint

    def __download(self, year: int, **read_kwargs)->pd.DataFrame:
        """
        Downloads the LOA file for a given year and returns it as a pandas DataFrame.

        Args:
            year (int): The year for which the LOA file is to be downloaded.

        Returns:
            pd.DataFrame: The downloaded LOA file as a pandas DataFrame.
        """



        url = self.__build_url(year)
        df = download_csv_as_dataframe(url, **read_kwargs)
        
        return df

    def __concat_year_dfs(self, year_list:[str], **read_kwargs)->pd.DataFrame:
        """
        Concatenates DataFrames for the specified years into a single DataFrame.

        Args:
            year_list (list): List of years for which DataFrames are to be concatenated.

        Returns:
            pd.DataFrame: The concatenated DataFrame.
        """
        dfs = [self.__download(year, **read_kwargs) for year in year_list]

        return pd.concat(dfs, ignore_index=True)

    def __describe_concated_df(self, df: pd.DataFrame) -> None:
        """
        Prints the shape, columns, and missing values for the given DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to describe.
        """
        print("Shape of DataFrame:", df.shape)
        print("Columns in DataFrame:", df.columns.tolist())
        print("Missing values per column:")
        print(df.isnull().sum())


    def __call__(self, year_list: Optional[[str]]=None, **read_kwargs)->pd.DataFrame:
        """
        Downloads and concatenates LOA files for the specified years.

        Args:
            year_list (list): List of years for which LOA files are to be downloaded.

        Returns:
            pd.DataFrame: The concatenated DataFrame containing the LOA data.
        """
        if year_list is None:
            year_list = self.files.keys()

        df = self.__concat_year_dfs(year_list, **read_kwargs)
        self.__describe_concated_df(df)

        return df

In [3]:
download_da = DownloadLoaRegionalizada()

In [4]:
df = download_da(encoding='latin-1', sep=';')

Shape of DataFrame: (62117, 22)
Columns in DataFrame: ['COD_PROGRAMA_METAS', 'COD_COMPLT_DA', 'COD_META', 'DESC_META', 'COD_REGIAO', 'DESC_REGIAO', 'COD_SUBPREFEITURA', 'DESC_SUBPREFEITURA', 'COD_DISTRITO', 'DESC_DISTRITO', 'COD_DA', 'DESC_DA', 'VALOR_DA', 'ORGAO', 'DESC_ORGAO', 'UNIDADE', 'DESC_UNIDADE', 'PA', 'DESC_PA', 'ANO_EX', 'COD_EMP', 'NOME_EMP']
Missing values per column:
COD_PROGRAMA_METAS    0
COD_COMPLT_DA         0
COD_META              0
DESC_META             0
COD_REGIAO            0
DESC_REGIAO           0
COD_SUBPREFEITURA     0
DESC_SUBPREFEITURA    0
COD_DISTRITO          0
DESC_DISTRITO         0
COD_DA                0
DESC_DA               0
VALOR_DA              0
ORGAO                 0
DESC_ORGAO            0
UNIDADE               0
DESC_UNIDADE          0
PA                    0
DESC_PA               0
ANO_EX                0
COD_EMP               0
NOME_EMP              0
dtype: int64


In [5]:
df.head()

,COD_PROGRAMA_METAS,COD_COMPLT_DA,COD_META,DESC_META,COD_REGIAO,DESC_REGIAO,COD_SUBPREFEITURA,DESC_SUBPREFEITURA,COD_DISTRITO,DESC_DISTRITO,COD_DA,DESC_DA,VALOR_DA,ORGAO,DESC_ORGAO,UNIDADE,DESC_UNIDADE,PA,DESC_PA,ANO_EX,COD_EMP,NOME_EMP
0,4,777.01.01.00.0001,777,Despesa Regionalizável,1,Norte,1,Subprefeitura Perus/Anhanguera,0,Supra-Distrital,1,Subprefeitura Perus/Anhanguera,"15631304,34",27,Secretaria Municipal do Verde e do Meio Ambiente,10,Gabinete do Secretário,2703,Manutenção e Operação de Parques Urbanos e Lin...,2025,1,PREFEITURA DO MUNICÍPIO DE SÃO PAULO
1,4,777.01.02.00.0001,777,Despesa Regionalizável,1,Norte,2,Subprefeitura Pirituba/Jaraguá,0,Supra-Distrital,1,Subprefeitura Pirituba/Jaraguá,"11067455,85",27,Secretaria Municipal do Verde e do Meio Ambiente,10,Gabinete do Secretário,2703,Manutenção e Operação de Parques Urbanos e Lin...,2025,1,PREFEITURA DO MUNICÍPIO DE SÃO PAULO
2,4,777.04.13.00.0001,777,Despesa Regionalizável,4,Sul,13,Subprefeitura Ipiranga,0,Supra-Distrital,1,Subprefeitura Ipiranga,0,27,Secretaria Municipal do Verde e do Meio Ambiente,10,Gabinete do Secretário,7129,"Ampliação, Reforma e Requalificação de Viveiro...",2025,1,PREFEITURA DO MUNICÍPIO DE SÃO PAULO
3,4,777.02.21.00.0001,777,Despesa Regionalizável,2,Leste,21,Subprefeitura Penha,0,Supra-Distrital,1,Subprefeitura Penha,0,27,Secretaria Municipal do Verde e do Meio Ambiente,10,Gabinete do Secretário,7129,"Ampliação, Reforma e Requalificação de Viveiro...",2025,1,PREFEITURA DO MUNICÍPIO DE SÃO PAULO
4,4,777.02.21.00.0001,777,Despesa Regionalizável,2,Leste,21,Subprefeitura Penha,0,Supra-Distrital,1,Subprefeitura Penha,0,29,Secretaria Municipal de Urbanismo e Licenciamento,20,Operação Urbana Consorciada Água Branca,3350,"Ampliação, Reforma e Requalificação de Áreas P...",2025,1,PREFEITURA DO MUNICÍPIO DE SÃO PAULO


### Padronizando cols

In [6]:
df.columns

Index(['COD_PROGRAMA_METAS', 'COD_COMPLT_DA', 'COD_META', 'DESC_META',
       'COD_REGIAO', 'DESC_REGIAO', 'COD_SUBPREFEITURA', 'DESC_SUBPREFEITURA',
       'COD_DISTRITO', 'DESC_DISTRITO', 'COD_DA', 'DESC_DA', 'VALOR_DA',
       'ORGAO', 'DESC_ORGAO', 'UNIDADE', 'DESC_UNIDADE', 'PA', 'DESC_PA',
       'ANO_EX', 'COD_EMP', 'NOME_EMP'],
      dtype='object')

In [7]:
colunas_padronizadas = {
    'ANO_EX' : 'exercicio',
    'ORGAO' : 'cod_orgao',
    'UNIDADE' : 'cod_unidade',
    'PA' : 'cod_proj_atividade',
    'COD_REGIAO' : 'cod_regiao',
    'COD_SUBPREFEITURA' : 'cod_subprefeitura',
    'COD_DISTRITO' : 'cod_distrito',
    'DESC_REGIAO' : 'desc_regiao',
    'DESC_SUBPREFEITURA' : 'desc_subprefeitura',
    'DESC_DISTRITO' : 'desc_distrito',
    'VALOR_DA' : 'valor_regionalizado'
}

In [8]:
df = df[[col for col in colunas_padronizadas.keys()]]

In [9]:
df = df.rename(columns=colunas_padronizadas)

In [10]:
df.head()

,exercicio,cod_orgao,cod_unidade,cod_proj_atividade,cod_regiao,cod_subprefeitura,cod_distrito,desc_regiao,desc_subprefeitura,desc_distrito,valor_regionalizado
0,2025,27,10,2703,1,1,0,Norte,Subprefeitura Perus/Anhanguera,Supra-Distrital,"15631304,34"
1,2025,27,10,2703,1,2,0,Norte,Subprefeitura Pirituba/Jaraguá,Supra-Distrital,"11067455,85"
2,2025,27,10,7129,4,13,0,Sul,Subprefeitura Ipiranga,Supra-Distrital,0
3,2025,27,10,7129,2,21,0,Leste,Subprefeitura Penha,Supra-Distrital,0
4,2025,29,20,3350,2,21,0,Leste,Subprefeitura Penha,Supra-Distrital,0


### Arrumando datatypes

In [11]:
df.dtypes

exercicio               int64
cod_orgao               int64
cod_unidade             int64
cod_proj_atividade      int64
cod_regiao              int64
cod_subprefeitura       int64
cod_distrito            int64
desc_regiao            object
desc_subprefeitura     object
desc_distrito          object
valor_regionalizado    object
dtype: object

In [12]:
df['valor_regionalizado'].value_counts(dropna=False)

valor_regionalizado
0          21697
0          11123
1000        1428
31          1171
31,25        688
           ...  
9840650        1
3591224        1
4116481        1
3560000        1
629644         1
Name: count, Length: 10035, dtype: int64

In [13]:
df['valor_regionalizado'].head(3)

0    15631304,34
1    11067455,85
2              0
Name: valor_regionalizado, dtype: object

In [14]:
df['valor_regionalizado'] = df['valor_regionalizado'].astype(str).str.replace(',', '.').astype(float)

In [15]:
df['valor_regionalizado'].isnull().any()

np.False_

In [16]:
df['valor_regionalizado'].describe()

count    6.211700e+04
mean     6.699377e+06
std      1.125534e+08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      3.743770e+05
max      1.223759e+10
Name: valor_regionalizado, dtype: float64

In [17]:
df.dtypes

exercicio                int64
cod_orgao                int64
cod_unidade              int64
cod_proj_atividade       int64
cod_regiao               int64
cod_subprefeitura        int64
cod_distrito             int64
desc_regiao             object
desc_subprefeitura      object
desc_distrito           object
valor_regionalizado    float64
dtype: object

### criando o indice para dar merge

indice igual no outro notebook

In [18]:
def fill_zeros(df:pd.DataFrame, col:str, num_zeros:int)->None:
    """
    Fills NaN values in the specified column of the DataFrame with zeros.

    Args:
        df (pd.DataFrame): The DataFrame to modify.
        col (str): The column name to fill with zeros.
    """
    df[col] = df[col].astype(int).astype(str)
    df[col] = df[col].str.zfill(num_zeros)

    print(col, df[col].str.len().unique())

In [19]:
fill_zeros(df, 'exercicio', 4)
fill_zeros(df, 'cod_orgao', 2)
fill_zeros(df, 'cod_unidade', 2)
fill_zeros(df, 'cod_proj_atividade', 4)

exercicio [4]
cod_orgao [2]
cod_unidade [2]
cod_proj_atividade [4]


In [20]:
df['index'] = df[['exercicio', 'cod_orgao', 'cod_unidade', 'cod_proj_atividade']].astype(str).agg('.'.join, axis=1)

In [21]:
df.columns

Index(['exercicio', 'cod_orgao', 'cod_unidade', 'cod_proj_atividade',
       'cod_regiao', 'cod_subprefeitura', 'cod_distrito', 'desc_regiao',
       'desc_subprefeitura', 'desc_distrito', 'valor_regionalizado', 'index'],
      dtype='object')

In [22]:
df.drop(['exercicio', 'cod_orgao', 'cod_unidade', 'cod_proj_atividade'], axis=1, inplace=True)
df.head()

,cod_regiao,cod_subprefeitura,cod_distrito,desc_regiao,desc_subprefeitura,desc_distrito,valor_regionalizado,index
0,1,1,0,Norte,Subprefeitura Perus/Anhanguera,Supra-Distrital,15631304.34,2025.27.10.2703
1,1,2,0,Norte,Subprefeitura Pirituba/Jaraguá,Supra-Distrital,11067455.85,2025.27.10.2703
2,4,13,0,Sul,Subprefeitura Ipiranga,Supra-Distrital,0.00,2025.27.10.7129
3,2,21,0,Leste,Subprefeitura Penha,Supra-Distrital,0.00,2025.27.10.7129
4,2,21,0,Leste,Subprefeitura Penha,Supra-Distrital,0.00,2025.29.20.3350


In [23]:
df['index'].isnull().any()

np.False_

In [24]:
df['index'].duplicated().mean()

np.float64(0.869777999581435)

In [25]:
df.to_csv(os.path.join(DATA_FOLDER, 'df_da.csv'), index=False, sep=';')